# Limpieza de Datos

### Importamos las Librerias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')

### Importamos los Datos

In [8]:
# importing dataset
location = r"./properati.csv" ## un '.' busca en la carpeta que lo contiene
data=pd.read_csv(location,sep=',')

ParserError: Error tokenizing data. C error: Expected 26 fields in line 216, saw 32


### Analizamos la estructura del Dataframe

In [6]:
#Resumir el Dataset
def Summary(data):
    data_null=data.isnull().sum()
    data_Not_Null=data.notnull().sum()
    data_total=data.shape[0]
    data_id=data.index
    data_null_porc=(data_null*100/data_total).round(2)
    data_type=data.dtypes
#     data_min=data.min()
#     data_mean=data.mean()
#     data_max=data.max()

    dataSumm=pd.DataFrame({'Tipo Dato':data_type,
                        'No Nulos':data_Not_Null,
                       'Nulos':data_null,
                       'Total':data_total,
                        '% Nulos':data_null_porc,
#                         'Minimo':data_min,
#                         'Media':data_mean,
#                         'Maximo':data_max
                          }).sort_values(by='% Nulos',ascending=False)
    return dataSumm

In [7]:
# #Genero el resumen en Excel
Summary(data)
# Summary(data).to_excel('Summary.xlsx',sheet_name='Hoja1')

,Tipo Dato,No Nulos,Nulos,Total,% Nulos
floor,float64,7899,113321,121220,93.48
expenses,float64,14262,106958,121220,88.23
rooms,float64,47390,73830,121220,60.91
price_usd_per_m2,float64,68617,52603,121220,43.39
lat-lon,object,69670,51550,121220,42.53
lat,float64,69670,51550,121220,42.53
lon,float64,69670,51550,121220,42.53
surface_total_in_m2,float64,81892,39328,121220,32.44
price_per_m2,float64,87658,33562,121220,27.69
price_aprox_local_currency,float64,100810,20410,121220,16.84


Eliminamos los Campos con mas del 60% Nulos que no tienen relevancia para el analisis

In [ ]:
# Eliminamos campos con mas de 60% de Nulos
data_mdf=data.copy()
data_mdf.drop(columns=['expenses','Unnamed: 0'],inplace=True)

In [ ]:
#Controlamos como quedo.
Summary(data_mdf)

SE PUEDE ELIMINAR
    UNNAMED 0
    PLACE_WITH_PARENTS_NAMES (ES CONCATENACION DE OTROS CAMPOS YA EXISTENTES)
REVISAR RELACIONES:
    METROS 2 DE SUPERFICIE NO PUEDEN SER MAYORES A LOS DE LO EDIFICADO
    TIPO DE CAMBIO APLICADO ENTRE PESOS Y USD
   
    
    

## Variables Numericas

### Controlando los Campos Numericos

**1.1 
Controlamos las Equivalencias entre currency y valor usd aprox**

In [ ]:
def price_fields(df,Curr):
    mask = df['currency'] == Curr
    a = all(df.loc[mask,'price']== df.loc[mask,'price_aprox_usd'])
    return print('Todos los valores de price con currency USD son iguales a los de price_apox_usd?',str(a))


In [ ]:
price_fields(data_mdf,'USD')


**2.1 CAMPOS SUPERFICIE METROS CUADRADOS CUBIERTA Y TOTAL**

**2.1.2 Vamos a establecer si la la relacion entre SUP CUBIERTA Y SUP TOTAL es razonable**

In [ ]:
#Crear mascara para verificar si esta correcto la relacion Sup cubierta vs Sup total
mask_M2_dif=(data_mdf['surface_covered_in_m2'])>\
            (data_mdf['surface_total_in_m2'])
# all(data_mdf)
print("Cantidad de Registros a Corregir",data_mdf.loc[mask_M2_dif].shape[0])

In [ ]:
#Corregimos la inconcistencia
data_mdf.loc[mask_M2_dif,['surface_total_in_m2']]=data_mdf['surface_covered_in_m2']

#chequeamos Nuevamente
mask_M2_dif=(data_mdf['surface_covered_in_m2'])>\
            (data_mdf['surface_total_in_m2'])
# all(data_mdf)
print("Cantidad de Registros a Corregir",data_mdf.loc[mask_M2_dif].shape[0])

**2.1.1 Vamos a imputar los valores de SUP Cubierta a Sup Total en los casos donde este ultimo es Null**

In [ ]:
#Crear mascara para verificar cuantos Nulos tienen SUP TOTAL que si tengan valores SUP CUB
mask_M2_null=(data_mdf['surface_total_in_m2'].isnull())&\
                (data_mdf['surface_covered_in_m2'].notnull())

print("Existen la cantidad de",data_mdf[mask_M2_null].shape[0], "registros en SUP TOTAL como Nulos que Tienen Valores en SUP CUB")



In [ ]:
#Vamos a llenar con los Valores de M2 Sup Cubierta los valores Faltantes de Sup Total
#Crear mascara para verificar cuales son los que son Null en Sup Total

mask_M2_null=(data_mdf['surface_total_in_m2'].isnull())&\
                    (data_mdf['surface_covered_in_m2'].notnull())

data_mdf.loc[mask_M2_null,['surface_total_in_m2','surface_covered_in_m2']]

#vamos a Copiar el DF para trabajar sobre uno nuevo
data_mdf_M2_OK=data_mdf.copy()

data_mdf_M2_OK['surface_total_in_m2'].fillna(value=data_mdf_M2_OK['surface_covered_in_m2'],inplace=True)


In [ ]:
print("Registros nulos en Surface Total antes de la imputación: ",data_mdf['surface_total_in_m2'].isnull().sum())
print("Regsitros nulos en Surface despues de la Imputación: ",data_mdf_M2_OK['surface_total_in_m2'].isnull().sum())

**2.1.3 Existen registros que pueden ser imputados capturando datos desde title**

In [ ]:
#paso a lower a todos los campos strings descriptivos
data_mdf_M2_OK['title']=data_mdf_M2_OK['title'].str.lower()
data_mdf_M2_OK['description']=data_mdf_M2_OK['description'].str.lower()

In [ ]:
#Creamos el regex
M2_captured=data_mdf_M2_OK['title'].str.extract('(?P<num>\d+\.?\d+)(\,)?(^\.)?(\d*)?(\s)?(?=m2|m²|mts|mts2|metros)')['num'].replace({'\.':""},regex=True).astype(float)

#Creo la mascara para controlar la cantidad que puede ser imputada
mask=(M2_captured.notnull())&(M2_captured!=False)&(data_mdf_M2_OK['surface_total_in_m2'].isnull())

print("Existen",data_mdf_M2_OK.loc[mask,:].shape[0],"registros que pueden ser imputados desde title")


In [ ]:
#Relleno los datos con lo capturado desde Title
#Campo Backup
data_mdf_M2_OK['ad_surface_m2_title']=M2_captured
data_mdf_M2_OK['surface_total_in_m2'].fillna(M2_captured,axis=0,inplace=True)

In [ ]:
#Controlo que ya no tenga mas datos para llenar
mask=(M2_captured.notnull())&(M2_captured!=False)&(data_mdf_M2_OK['surface_total_in_m2'].isnull())
print("Existen",data_mdf_M2_OK.loc[mask,:].shape[0],"registros que pueden ser imputados desde title")

**2.1.4 Existen registros que pueden ser imputados capturando datos desde Description**

In [ ]:
#Creamos el regex
M2_captured_2=data_mdf_M2_OK['description'].str.extract('(?P<num>\d+\.?\d+)(\,)?(^\.)?(\d*)?(\s)?(?=m2|m²|mts|mts2|metros)')['num'].replace({'\.':""},regex=True).astype(float)

#Creo la mascara para controlar la cantidad que puede ser imputada
mask=(M2_captured_2.notnull())&(M2_captured_2!=False)&(data_mdf_M2_OK['surface_total_in_m2'].isnull())

print("Existen",data_mdf_M2_OK.loc[mask,:].shape[0],"registros que pueden ser imputados desde Description")


In [ ]:
#Relleno los datos con lo capturado desde description
#Campo Backup
data_mdf_M2_OK['ad_surface_m2_descrip']=M2_captured_2
data_mdf_M2_OK['surface_total_in_m2'].fillna(M2_captured_2,axis=0,inplace=True)

In [ ]:
#Controlo que ya no tenga mas datos para llenar
mask=(M2_captured_2.notnull())&(M2_captured_2!=False)&(data_mdf_M2_OK['surface_total_in_m2'].isnull())
print("Existen",data_mdf_M2_OK.loc[mask,:].shape[0],"registros que pueden ser imputados desde title")

**2.2 CAMPOS PRECIO USD Y PRECIO USD M2**

**VAMOS A CONTROLAR SI EXISTE FORMA DE LLENAR EL PRECIO USD M2 QUE FALTAN USANDO PRECIO USD Y M2 SUP TOTAL**

**2.2.1 Es Posible Capturar precio en dolares del campo Title para llenar el campo Precio_USD_aprox**

In [ ]:
#Creo una mascara para capturar los valores precedidos de U$S o U$D
df_rebuilt=data_mdf_M2_OK.copy()
USD_capture=df_rebuilt['title'].str.extract('(u\$?s?d) (\d+[\.]*\d*[\.]*\d*)')[1].replace({'\.':""},regex=True).astype(float)


In [ ]:
#Compruebo la cantidad de campos que se podrian llenar (no nulos del campo nuevo y nulos del campo Precio USD Aprox)

mask=(USD_capture.notnull())&(df_rebuilt['price_aprox_usd'].isnull())

print("Existen",df_rebuilt.loc[mask,:].shape[0],"registros que pueden ser imputados desde Title")


In [ ]:
#Hacemos la imputacion para llenar los campos de Price USD aprox
df_rebuilt_v2=df_rebuilt.copy()
#creo Campo de seguimiento
df_rebuilt_v2['ad_price_aprox_usd_title']=USD_capture 
#Realizo la imputacion
df_rebuilt_v2['price_aprox_usd'].fillna(value=USD_capture,axis=0,inplace=True)
#Controlamos si se hizo correctamente
mask=(USD_capture.notnull())&(df_rebuilt_v2['price_aprox_usd'].isnull())
print("Existen",df_rebuilt_v2.loc[mask,:].shape[0],"registros que pueden ser imputados desde Title")



**Comparamos los cambios antes de la imputacion**

In [ ]:
Summary(df_rebuilt[['surface_total_in_m2','price_aprox_usd','price_usd_per_m2']])

**Comparamos los cambios despues de la imputacion**

In [ ]:
Summary(df_rebuilt_v2[['surface_total_in_m2','price_aprox_usd','price_usd_per_m2']])

**2.2.2 Es Posible Capturar precio en dolares del campo Description para llenar el campo Precio_USD_aprox**

In [ ]:
#Creo una mascara para capturar los valores precedidos de U$S o U$D

USD_capture_2=df_rebuilt_v2['description'].str.extract('(u\$?s?d) (\d+[\.]*\d*[\.]*\d*)')[1].replace({'\.':""},regex=True).astype(float)


In [ ]:
#Compruebo la cantidad de campos que se podrian llenar (no nulos del campo nuevo y nulos del campo Precio USD Aprox)

mask=(USD_capture_2.notnull())&(df_rebuilt_v2['price_aprox_usd'].isnull())

print("Existen",df_rebuilt_v2.loc[mask,:].shape[0],"registros que pueden ser imputados desde description")


In [ ]:
#Hacemos la imputacion para llenar los campos de Price USD aprox
#creo Campo de seguimiento
df_rebuilt_v2['ad_price_aprox_usd_descrip']=USD_capture       
#Realizo la imputacion
df_rebuilt_v2['price_aprox_usd'].fillna(value=USD_capture,axis=0,inplace=True)
#Controlo si se efectuo correctamente
mask=(USD_capture.notnull())&(df_rebuilt_v2['price_aprox_usd'].isnull())
print("Existen",df_rebuilt_v2.loc[mask,:].shape[0],"registros que pueden ser imputados desde description")



**Comparamos los cambios antes de la imputacion**

In [ ]:
Summary(df_rebuilt[['surface_total_in_m2','price_aprox_usd','price_usd_per_m2']])

**Comparamos los cambios despues de la imputacion**

In [ ]:
Summary(df_rebuilt_v2[['surface_total_in_m2','price_aprox_usd','price_usd_per_m2']])

**2.2.2 Ahora podemos tomar Precio USD Aprox para rellenar Precio USD M2**

In [ ]:
#Creo una Mascara para poder determinar los registros en donde Precio USD x M2 siendo nulos tiene valores utilizables en Precio USD y Sup Total 
mask_price_x_m2_null=\
(df_rebuilt_v2['price_usd_per_m2'].isnull())&\
(df_rebuilt_v2['price_aprox_usd'].notnull())&\
(df_rebuilt_v2['surface_total_in_m2'].notnull())

print("Existen",data_mdf_M2_OK.loc[mask_price_x_m2_null,:].shape[0],"registros que pueden ser Imputados")

In [ ]:
#ahora lleno los valores nulos con los datos 
#Creo copia del DF
df_rebuilt_v3=df_rebuilt_v2.copy()
#Lleno con apply

Price_USD_M2_calculated=df_rebuilt_v3.apply(lambda x: 0 if x['surface_total_in_m2']==0 else x['price_aprox_usd'] / x['surface_total_in_m2'],axis=1)
df_rebuilt_v3['price_usd_per_m2']=Price_USD_M2_calculated

#compruebo
mask_price_x_m2_Notnull=\
(df_rebuilt_v3['price_usd_per_m2'].isnull())&\
(df_rebuilt_v3['price_aprox_usd'].notnull()&\
 df_rebuilt_v3['surface_total_in_m2'].notnull())
print("Existian valores Nulos para imputar?: ",any(mask_price_x_m2_null))
print("Existen valores Nulos para imputar ahora que hicimos la imputacion?: ",any(mask_price_x_m2_Notnull))

**2.2.3 Comparamos el efecto de la imputacion en los campos respectivos**

In [ ]:
Summary(df_rebuilt[['surface_total_in_m2','price_aprox_usd','price_usd_per_m2','state_name','property_type']])

In [ ]:
Summary(df_rebuilt_v2[['surface_total_in_m2','price_aprox_usd','price_usd_per_m2','state_name','property_type']])

In [ ]:
Summary(df_rebuilt_v3[['surface_total_in_m2','price_aprox_usd','price_usd_per_m2','state_name','property_type']])

**Nota:** Resta evaluar si se puede llenar el Campo Precio USD M2 por medio de los promedios de los barrios

**2.2.4 Evaluamos la informacion agrupada por Barrio y Tipo de propiedad**

In [ ]:
df_rebuilt_v3.groupby(['property_type','place_name','rooms'])['surface_total_in_m2'].describe()

**Elimino los Campos que no vamos a usar**

In [ ]:
df_rebuilt_v3.drop(columns=['price','price_aprox_local_currency','surface_covered_in_m2','price_per_m2','operation'],inplace=True)

**Elimino los elementos duplicados en la columna description**

In [ ]:
#calculo los duplicados
print("Existen",df_rebuilt_v3.duplicated(['description','title','property_type']).sum(),"Duplicados")

In [ ]:
df_rebuilt_v3.drop_duplicates(['description','title','property_type'],keep='last',inplace=True)

In [ ]:
#control de los duplicados
print("Existen",df_rebuilt_v3.duplicated(['description','title','property_type']).sum(),"Duplicados")

### Outliers

**Campo Surface M2 total**

In [ ]:
sns.boxplot(x = df_rebuilt_v3['surface_total_in_m2'])

In [ ]:
sns.distplot(df_rebuilt_v3['surface_total_in_m2'])

**Buscamos los Outliers de Superficie Total M2 por tipo de propiedad**

In [ ]:
#Version plana de Revision de outliers de Sup Total

df=df_rebuilt_v3.groupby(['property_type'])['surface_total_in_m2']

UR_limit=pd.DataFrame((df.quantile(0.75))+((df.quantile(0.75)-df.quantile(0.25))*1.5))
LR_limit=pd.DataFrame((df.quantile(0.05)))
    


UR_limit.columns=["UR_limit_M2"]
LR_limit.columns=["LR_limit_M2"]


df_rebuilt_v3=df_rebuilt_v3.merge(right=UR_limit, on=('property_type'))
df_rebuilt_v3=df_rebuilt_v3.merge(right=LR_limit, on=('property_type'))



**Imputamos desde lo capturado de title a los Sup M2 los que estan menos del 5% del percentil**

In [ ]:
mask_0=df_rebuilt_v3['surface_total_in_m2']<df_rebuilt_v3['LR_limit_M2']
mask_2=df_rebuilt_v3['ad_surface_m2_title'].notnull()
df_rebuilt_v3.loc[mask_0&mask_2,['surface_total_in_m2']]=df_rebuilt_v3['ad_surface_m2_title']

**Detectamos los Outliers**

In [ ]:
#Detecto los outliers
listCol=['property_type','surface_total_in_m2','ad_surface_m2_title','ad_surface_m2_descrip','UR_limit_M2','LR_limit_M2','description','title']
mask_out=(df_rebuilt_v3['surface_total_in_m2']>df_rebuilt_v3['UR_limit_M2'])|(df_rebuilt_v3['surface_total_in_m2']<df_rebuilt_v3['LR_limit_M2'])
df_rebuilt_v3.loc[mask_out,listCol].to_excel('outliers23.xlsx')


**Eliminamos los Outliers**

In [ ]:
#Eliminamos lor Outliers creando un DF Nuevo (v4)

df_rebuilt_v4=df_rebuilt_v3.loc[~mask_out]

print("El DF tiene",df_rebuilt_v3.shape[0],"registros antes de la eliminacion de outliers")
print("Los outliers son",df_rebuilt_v3.loc[mask_out].shape[0],"registros ")
print("El DF tiene",df_rebuilt_v4.shape[0],"registros despues de la eliminacion de outliers")

**Graficamos Nuevamente**

In [ ]:
sns.boxplot(x = df_rebuilt_v4['surface_total_in_m2'])

In [ ]:
sns.distplot(df_rebuilt_v4['surface_total_in_m2'])

**Buscamos los Outliers de Precio USD Aprox por tipo de propiedad y Zona**

In [ ]:
#Version plana de Revision de outliers de Precio USD Aprox

df=df_rebuilt_v4.groupby(['property_type','state_name'])['price_aprox_usd']

UR_limit=pd.DataFrame((df.quantile(0.75))+((df.quantile(0.75)-df.quantile(0.25))*1.5))
LR_limit=pd.DataFrame((df.quantile(0.05)))
    
UR_limit.columns=["UR_limit_USD"]
LR_limit.columns=["LR_limit_USD"]


df_rebuilt_v4=df_rebuilt_v4.merge(right=UR_limit, on=(['property_type','state_name']))
df_rebuilt_v4=df_rebuilt_v4.merge(right=LR_limit, on=(['property_type','state_name']))



In [ ]:
#Detecto los outliers

mask_out=(df_rebuilt_v4['price_aprox_usd']>df_rebuilt_v4['UR_limit_USD'])|((df_rebuilt_v4['price_aprox_usd']<df_rebuilt_v4['LR_limit_USD']))


In [ ]:
#Elimino los Outliers de Precio USD Aprox
df_rebuilt_v5=df_rebuilt_v4.loc[~mask_out]

print("El DF tiene",df_rebuilt_v4.shape[0],"registros antes de la eliminacion de outliers")
print("Los outliers son",df_rebuilt_v4.loc[mask_out].shape[0],"registros ")
print("El DF tiene",df_rebuilt_v5.shape[0],"registros despues de la eliminacion de outliers")

In [ ]:
sns.boxplot(x = df_rebuilt_v5['price_aprox_usd'])

In [ ]:
sns.distplot(df_rebuilt_v5['price_aprox_usd'])

**Buscamos los Outliers de Precio USD M2 por tipo de propiedad y Zona**

**Aplicamos la Division a todos los Registros Price USD M2**

In [ ]:
Price_USD_M2_Cal=df_rebuilt_v5.apply(lambda x: 0 if x['surface_total_in_m2']==0 else x['price_aprox_usd'] / x['surface_total_in_m2'],axis=1)
df_rebuilt_v5['price_usd_per_m2']=Price_USD_M2_Cal


**Calculamos Outliers en Price USD M2**

In [ ]:
#Version plana de Revision de outliers de Precio USD M2

df=df_rebuilt_v5.groupby(['property_type','state_name'])['price_usd_per_m2']

UR_limit=pd.DataFrame((df.quantile(0.75))+((df.quantile(0.75)-df.quantile(0.25))*1.5))
LR_limit=pd.DataFrame((df.quantile(0.05)))
    
UR_limit.columns=["UR_limit_USDM2"]
LR_limit.columns=["LR_limit_USDM2"]


df_rebuilt_v5=df_rebuilt_v5.merge(right=UR_limit, on=(['property_type','state_name']))
df_rebuilt_v5=df_rebuilt_v5.merge(right=LR_limit, on=(['property_type','state_name']))



In [ ]:
#Detecto los outliers
listCol=['property_type','state_name','price_aprox_usd','ad_price_aprox_usd_title','ad_price_aprox_usd_descrip',\
         'UR_limit_USDM2','surface_total_in_m2','description','title']
mask_out=(df_rebuilt_v5['price_usd_per_m2']>df_rebuilt_v5['UR_limit_USDM2'])|(df_rebuilt_v5['price_usd_per_m2']<df_rebuilt_v5['LR_limit_USDM2'])
# df_rebuilt_v5.loc[mask_out,['property_type','state_name','price_usd_per_m2','UR_limit_USDM2','UR_limit_USDM2']].to_excel('usdM2.xlsx')

In [ ]:
#Elimino los Outliers de Precio USD M2
df_rebuilt_v6=df_rebuilt_v5.loc[~mask_out]

print("El DF tiene",df_rebuilt_v5.shape[0],"registros antes de la eliminacion de outliers")
print("Los outliers son",df_rebuilt_v5.loc[mask_out].shape[0],"registros ")
print("El DF tiene",df_rebuilt_v6.shape[0],"registros despues de la eliminacion de outliers")

In [ ]:
sns.boxplot(x = df_rebuilt_v6['price_usd_per_m2'])

In [ ]:
sns.distplot(df_rebuilt_v6['price_usd_per_m2'])

## Variables no Numericas

### Campos No Numericos

**Campos Categoricos**

Existen campos que deben ser considerados Categorical, como ser:
1. Property_type
2. State Name

In [ ]:
df_rebuilt_v6['property_type']=df_rebuilt_v6['property_type'].astype('category')
df_rebuilt_v6['state_name']=df_rebuilt_v6['state_name'].astype('category')

**Campo Place Name y State Name**

**Usando str.split crearemos campos control para poder contrastar contra los originales**

In [ ]:
df_rebuilt_v6[['eliminar','ad_country','ad_state_name','ad_place_name3','ad_place_name_4']]=df_rebuilt_v6['place_with_parent_names'].str.split('|',expand=True,n=4)    


In [ ]:
# df_rebuilt_v5[['place_with_parent_names','eliminar','ad_country','ad_state_name','ad_place_name3','ad_place_name_4','place_name']].to_excel('state.xlsx')

Verificamos que State name este correcto

In [ ]:
#State name es igual al que esta en el campo concatenado
#Copiamos el DF a uno nuevo
dfv5=df_rebuilt_v6.copy()
print("Son Iguales los campos State Name con los valores del parent field ?",all(dfv5['ad_state_name']==dfv5['state_name']))

Hacemos lo mismo con Place Name

In [ ]:
print("Son Iguales los campos place Name con los valores del parent field ?",all(dfv5['ad_place_name3']==dfv5['place_name']))

In [ ]:
#vemos donde esta la diferencia
#mascara de la diferencia entre las dos columnas
mask_diff=(dfv5['ad_place_name3']!=dfv5['place_name'])
#aplico el cambio
dfv5.loc[mask_diff,['place_name']]=dfv5['ad_place_name3']

In [ ]:
all(mask_diff)

Volvemos a Controlar

In [ ]:
print("Son Iguales los campos place Name con los valores del parent field ?",all(dfv5['ad_place_name3']==dfv5['place_name']))

Eliminamos los campos de Control

In [ ]:
dfv5.drop(columns=['ad_place_name_4','ad_place_name3','ad_state_name','ad_country','ad_price_aprox_usd_title',\
                   'ad_price_aprox_usd_descrip','ad_surface_m2_title','ad_surface_m2_descrip','UR_limit_USD','UR_limit_M2','LR_limit_USD','LR_limit_M2','currency','floor','eliminar'],inplace=True)

In [ ]:
Summary(dfv5.select_dtypes(exclude=float))

In [ ]:
Summary(dfv5.select_dtypes(include=float))

In [ ]:
listColum=['property_type','place_name','state_name','price_usd_per_m2','price_aprox_usd','surface_total_in_m2','rooms']
Summary(dfv5[listColum])

**Campo Rooms**

Vamos a Revisar si existe forma de capturar datos de Descripcion

**Capturo datos desde Description**

In [ ]:
#Creo una mascara para capturar los valores de Rooms

Mask_Rooms=dfv5['description'].str.extract('(?P<rooms>\d)(\s*?)(amb|ambiente|ambientes|habitaciones|dormitorios|piezas)')['rooms'].astype(float)

#Compruebo la cantidad de campos que se podrian llenar (no nulos del campo nuevo y nulos del campo Rooms)

mask=(Mask_Rooms.notnull())&(dfv5['rooms'].isnull())

print("Existen",dfv5.loc[mask,:].shape[0],"registros que pueden ser imputados desde description")


In [ ]:
#Hacemos la imputacion para llenar los campos de Price USD aprox

#creo Campo de seguimiento
dfv5['ad_Rooms_descrip']=Mask_Rooms 
#Realizo la imputacion
dfv5['rooms'].fillna(value=Mask_Rooms,axis=0,inplace=True)
#Controlamos si se hizo correctamente
mask=(Mask_Rooms.notnull())&(dfv5['rooms'].isnull())
print("Existen",dfv5.loc[mask,:].shape[0],"registros que pueden ser imputados desde description")



**Capturo datos desde title**

In [ ]:
#Creo una mascara para capturar los valores de Rooms

Mask_Rooms=dfv5['title'].str.extract('(?P<rooms>\d)(\s*?)(amb|ambiente|ambientes|habitaciones|dormitorios|piezas)')['rooms'].astype(float)


#Compruebo la cantidad de campos que se podrian llenar (no nulos del campo nuevo y nulos del campo Rooms)

mask=(Mask_Rooms.notnull())&(dfv5['rooms'].isnull())

print("Existen",dfv5.loc[mask,:].shape[0],"registros que pueden ser imputados desde title")


In [ ]:
#Hacemos la imputacion para llenar los campos de Price USD aprox

#creo Campo de seguimiento
dfv5['ad_Rooms_title']=Mask_Rooms 
#Realizo la imputacion
dfv5['rooms'].fillna(value=Mask_Rooms,axis=0,inplace=True)
#Controlamos si se hizo correctamente
mask=(Mask_Rooms.notnull())&(dfv5['rooms'].isnull())
print("Existen",dfv5.loc[mask,:].shape[0],"registros que pueden ser imputados desde title")



In [ ]:
#eliminamos campos de control
dfv5.drop(columns=['ad_Rooms_descrip','ad_Rooms_title'],inplace=True)

**Eliminamos Outliers de Rooms**

In [ ]:
#Version plana de Revision de outliers de Rooms

df=dfv5.groupby(['property_type','state_name'])['rooms']

UR_limit=pd.DataFrame((df.quantile(0.75))+((df.quantile(0.75)-df.quantile(0.25))*1.5))
LR_limit=pd.DataFrame((df.quantile(0.05)))
    
UR_limit.columns=["UR_limit_rooms"]
LR_limit.columns=["LR_limit_rooms"]


dfv5=dfv5.merge(right=UR_limit, on=(['property_type','state_name']))
dfv5=dfv5.merge(right=LR_limit, on=(['property_type','state_name']))



In [ ]:
#Detecto los outliers
listCol=['property_type','state_name','rooms','UR_limit_rooms','LR_limit_rooms']

mask_out=(dfv5['rooms']>dfv5['UR_limit_rooms'])|(dfv5['rooms']<dfv5['LR_limit_rooms'])

dfv5.loc[mask_out,['property_type','state_name','rooms','UR_limit_rooms','LR_limit_rooms']].to_excel('rooms.xlsx')

In [ ]:
#Elimino los Outliers de Precio USD M2
dfv6=dfv5.loc[~mask_out]

print("El DF tiene",dfv5.shape[0],"registros antes de la eliminacion de outliers")
print("Los outliers son",dfv5.loc[mask_out].shape[0],"registros ")
print("El DF tiene",dfv6.shape[0],"registros despues de la eliminacion de outliers")


In [ ]:
dfv6.drop(columns=['UR_limit_rooms','LR_limit_USDM2','LR_limit_rooms','UR_limit_USDM2'],inplace=True)

In [ ]:
dfv6['price_usd_per_m2'].isnull().sum()

In [ ]:
dfv6['price_usd_per_m2'].describe()

### Completamos faltantes con la media

In [ ]:
dfv7 = dfv6.copy()
#dfv7_group = dfv7.groupby(["property_type", "state_name", "place_name"])
#dfv7['price_usd_per_m2'] = dfv7_group["price_usd_per_m2"].transform(lambda grp: grp.fillna(grp.mean()))
#dfv7['price_aprox_usd'] = dfv7_group["price_aprox_usd"].transform(lambda grp: grp.fillna(grp.mean()))
#dfv7['surface_total_in_m2'] = dfv7_group["surface_total_in_m2"].transform(lambda grp: grp.fillna(grp.mean()))


In [ ]:
#dfv7.price_usd_per_m2.isnull().sum()

In [ ]:
#dfv7['price_usd_per_m2'].describe()

In [ ]:
Summary(dfv6)

## Generamos un csv Limpio para Relaciones y Graficos

### Borramos Nulos

In [ ]:
dfv8 = dfv7.copy()
#mask = , 'price_aprox_usd', 'surface_total_in_m2'
#subset=['name', 'born']
#subset=['name', 'born']

dfv8.dropna(subset=['price_usd_per_m2','price_aprox_usd', 'surface_total_in_m2'], inplace=True, axis=0)
dfv8.shape

In [ ]:
dfv7.shape

In [ ]:
dfv7['price_aprox_usd'].isnull().sum()

In [ ]:
dfv7['surface_total_in_m2'].isnull().sum()

## Agregamos Amenities

In [ ]:
dfv8['parrilla']=dfv8['description'].str.extract('(parrilla)').replace('parrilla', 1).astype(float)
dfv8['pileta']=dfv8['description'].str.extract('(pileta)').replace('pileta', 1).astype(float)
dfv8['cochera']=dfv8['description'].str.extract('(cochera)').replace('cochera', 1).astype(float)
dfv8['gimnasio']=dfv8['description'].str.extract('(gimnasio)').replace('gimnasio', 1).astype(float)

dfv8['cant_amenities'] = dfv8['parrilla'].fillna(0) + dfv8['pileta'].fillna(0) + dfv8['cochera'].fillna(0) + dfv8['gimnasio'].fillna(0)
dfv8['cant_amenities'].value_counts()

**Generacion del csv limpio**

In [ ]:
dfv8.to_csv("../Data/properati_clean.csv",sep=',')

In [ ]:
Summary(dfv8)

### Usando profiling

In [ ]:
prof=ProfileReport(dfv8,title='properati Report v8')
prof.to_file(output_file='output_dfv8.html')

In [ ]:
# prof=ProfileReport(data,title='properati Report v1')
# prof.to_file(output_file='output_data.html')

In [ ]:
#prof=ProfileReport(dfv6,title='properati Report v7')
#prof.to_file(output_file='output_dfv7.html')